Solving the same problem with Greedy Best First Search

In [27]:
from search_utils import AbstractNode, reconstruct_path
from typing import Callable, NamedTuple, Union


class GameState(NamedTuple):
    player_a: int
    player_b: int
    player_c: int

    def __repr__(self) -> str:
        return f"GameState(player A={self.player_a}, player B={self.player_b}, player C={self.player_c})"

    def __str__(self) -> str:
        return self.__repr__()

    def __eq__(self, __value: object) -> bool:
        if not isinstance(__value, GameState):
            return False

        return (
            self.player_a == __value.player_a
            and self.player_b == __value.player_b
            and self.player_c == __value.player_c
        )

    def __hash__(self) -> int:
        return hash((self.player_a, self.player_b, self.player_c))

    def is_within_boundaries(self, goal: "GameState") -> bool:
        return (
            self.player_a <= goal.player_a
            and self.player_b <= goal.player_b
            and self.player_c <= goal.player_c
        )

    def copy_with(self, **kwargs):
        return GameState(**{**self._asdict(), **kwargs})


class Node(AbstractNode[GameState]):
    def __init__(
        self,
        action: str,
        state: GameState,
        parent: Union[GameState, None] = None,
    ) -> None:
        super().__init__(action, state, parent)

    def __eq__(self, __value: object) -> bool:
        if not isinstance(__value, Node):
            return False

        return self.state == __value.state

In [28]:
CandidateAction = tuple[str, GameState]


def create_candidates(previous_state: GameState):
    A, B, C = previous_state

    candidates: list[CandidateAction] = [
        ("A plays with B", previous_state.copy_with(player_a=A + 1, player_b=B + 1)),
        ("A plays with C", previous_state.copy_with(player_a=A + 1, player_c=C + 1)),
        ("B plays with C", previous_state.copy_with(player_b=B + 1, player_c=C + 1)),
    ]

    return candidates

In [29]:
HeuristicFunction = Callable[[GameState], float]


def geometric_mean(state: GameState) -> float:
    A, B, C = state
    return (A * B * C) ** (1 / 3)

In [30]:
class PingPong:
    def __init__(
        self,
        initial_state: GameState,
        goal: GameState,
        heuristic: HeuristicFunction,
    ) -> None:
        self.initial_state = initial_state
        self.goal = goal
        self.heuristic = heuristic

    def neighbors(self, node: Node):
        candidates = create_candidates(node.state)
        neighbors: list[Node] = []

        for action, state in candidates:
            if state.is_within_boundaries(self.goal) and node.action != action:
                neighbors.append(Node(action, state, node))

        # On retourne les noeuds triés dans l'ordre décroissant du coût heuristique
        return sorted(neighbors, key=lambda n: self.heuristic(n.state))

    def solve(self):
        num_explored = 0
        explored: list[GameState] = list()
        frontier: list[Node] = []

        frontier.append(Node("START", self.initial_state))

        while frontier:
            current_node = frontier.pop(0)
            num_explored += 1

            print(
                f"Itération N°{num_explored} - Action - {current_node.action} - {current_node.state}"
            )

            if current_node.state == self.goal:
                return reconstruct_path(current_node), num_explored

            explored.append(current_node.state)
            for neighbor in self.neighbors(current_node):
                if neighbor.state not in explored and neighbor not in frontier:
                    frontier.append(neighbor)

        raise Exception("No solution found")

In [31]:
initial_state = GameState(0, 0, 0)
goal = GameState(10, 15, 17)
problem = PingPong(initial_state, goal, geometric_mean)
solution, num_explored = problem.solve()

Itération N°1 - Action - START - GameState(player A=0, player B=0, player C=0)
Itération N°2 - Action - A plays with B - GameState(player A=1, player B=1, player C=0)
Itération N°3 - Action - A plays with C - GameState(player A=1, player B=0, player C=1)
Itération N°4 - Action - B plays with C - GameState(player A=0, player B=1, player C=1)
Itération N°5 - Action - A plays with C - GameState(player A=2, player B=1, player C=1)
Itération N°6 - Action - B plays with C - GameState(player A=1, player B=2, player C=1)
Itération N°7 - Action - B plays with C - GameState(player A=1, player B=1, player C=2)
Itération N°8 - Action - A plays with B - GameState(player A=3, player B=2, player C=1)
Itération N°9 - Action - B plays with C - GameState(player A=2, player B=2, player C=2)
Itération N°10 - Action - A plays with B - GameState(player A=2, player B=3, player C=1)
Itération N°11 - Action - A plays with C - GameState(player A=2, player B=1, player C=3)
Itération N°12 - Action - A plays with 

Exception: No solution found

In [ ]:
actions = "\n".join([node.action for node in solution])
print(f"Solution:\n{actions}")

Solution:
START
A plays with B
A plays with B
A plays with B
A plays with B
A plays with C
A plays with C
A plays with C
A plays with C
A plays with C
A plays with C
B plays with C
B plays with C
B plays with C
B plays with C
B plays with C
B plays with C
B plays with C
B plays with C
B plays with C
B plays with C
B plays with C
